In [10]:
# use pyproject group: experiments_analysis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import yaml

ModuleNotFoundError: No module named 'pandas'

In [11]:
# Load YAML file
with open('../../results.yaml') as file:
    data = yaml.safe_load(file)

# Process metrics per experiment call
rows = []
for experiment in data['experiments']:
    for result in experiment['results']:
        metrics = result['metrics']
        # Annotate metrics with experiment name and date
        row = {'experiment': experiment['name'], 'date': data['date'], **metrics}
        rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)
print(df)


NameError: name 'yaml' is not defined

In [ ]:
# Raw data and basic setup
data = np.random.randn(10)
hist, bins = np.histogram(data, bins=10, density=True)
bin_centers = (bins[:-1] + bins[1:]) / 2

# Plot Histogram
plt.hist(data, bins=10, density=True, alpha=0.6, color='g', label='Histogram')

# PDF
pdf = stats.norm.pdf(bin_centers, np.mean(data), np.std(data))
plt.plot(bin_centers, pdf, label='PDF', color='red')

# CDF
#cdf = np.cumsum(hist*np.diff(bins))
#plt.plot(bin_centers, cdf, label='CDF', color='blue')

# Student's t-interval
t_interval = stats.t.interval(0.95, len(data)-1, loc=np.mean(data), scale=stats.sem(data))
plt.axvline(t_interval[0], color='orange', linestyle='--', label='95% t-interval')
plt.axvline(t_interval[1], color='orange', linestyle='--')

# Display
plt.legend()
plt.show()
